In [4]:
from glob import glob
from collections import defaultdict
import os, pandas as pd, numpy as np

ANNOTATION_BASE = "/Volumes/conflab-mm/v4/release/annotations/actions/laughing"
OUTPUT_DIR      = "./annotations/per_modality"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def load_annotation_files(modality):
    folder  = os.path.join(ANNOTATION_BASE, modality)
    pattern = os.path.join(folder, "**", "*.csv")
    files   = sorted(glob(pattern, recursive=True))
    print(f"[load] {modality}: found {len(files)} CSV files")
    return files

def consolidate_majority(modality, participant_ids=range(1,51)):
    seg_ann = defaultdict(list)
    files   = load_annotation_files(modality)

    for path in files:
        fname              = os.path.splitext(os.path.basename(path))[0]
        vid, seg, annotator = fname.rsplit("_", 2)
        print(f"[{modality}] loading → {vid}, {seg}, {annotator}")
        # …robust CSV read, cleaning, etc…
        df = pd.read_csv(path, encoding="latin1", engine="python", on_bad_lines="skip")
        seg_ann[(vid, seg)].append(df)

    print(f"[{modality}] grouped into {len(seg_ann)} (vid,seg) keys:", seg_ann.keys())

    records = []
    for (vid, seg), dfs in seg_ann.items():
        max_len = max(len(df) for df in dfs)
        for pid in participant_ids:
            votes = []
            for df in dfs:
                # turn whatever df.get() gives us into an ndarray
                arr = np.asarray(df.get(str(pid), np.zeros(len(df), dtype=int)))
                votes.append(np.pad(arr, (0, max_len - len(arr)), constant_values=0))
            majority = (np.stack(votes).sum(axis=0) >= 2).astype(int)
            for frame, label in enumerate(majority):
                records.append({
                    "modality":   modality,
                    "video_id":   vid,
                    "segment":    seg,
                    "participant":pid,
                    "frame":      frame,
                    "label":      label
                })

    out_df  = pd.DataFrame(records)
    out_path = os.path.join(OUTPUT_DIR, f"annotations_{modality.lower()}_majority.csv")
    out_df.to_csv(out_path, index=False)
    print(f"[save] {out_path}\n")

for modality in ["No_Audio", "Only_Audio", "With_Audio"]:
    consolidate_majority(modality)


[load] No_Audio: found 69 CSV files
[No_Audio] loading → vid2, seg7, ann1
[No_Audio] loading → vid2, seg7, ann2
[No_Audio] loading → vid2, seg7, ann3
[No_Audio] loading → vid2, seg8, ann1
[No_Audio] loading → vid2, seg8, ann2
[No_Audio] loading → vid2, seg8, ann3
[No_Audio] loading → vid2, seg9, ann1
[No_Audio] loading → vid2, seg9, ann2
[No_Audio] loading → vid2, seg9, ann3
[No_Audio] loading → vid3, seg1, ann1
[No_Audio] loading → vid3, seg1, ann2
[No_Audio] loading → vid3, seg1, ann3
[No_Audio] loading → vid3, seg2, ann1
[No_Audio] loading → vid3, seg2, ann2
[No_Audio] loading → vid3, seg2, ann3
[No_Audio] loading → vid3, seg3, ann1
[No_Audio] loading → vid3, seg3, ann2
[No_Audio] loading → vid3, seg3, ann3
[No_Audio] loading → vid3, seg4, ann1
[No_Audio] loading → vid3, seg4, ann2
[No_Audio] loading → vid3, seg4, ann3
[No_Audio] loading → vid3, seg5, ann1
[No_Audio] loading → vid3, seg5, ann2
[No_Audio] loading → vid3, seg5, ann3
[No_Audio] loading → vid3, seg6, ann1
[No_Audio] loa

In [5]:
ann_path = "./annotations/per_modality/annotations_only_audio_majority.csv"

df = pd.read_csv(ann_path)

for (vid, seg) in df.groupby(["video_id", "segment"]):
    print(vid)

('vid2', 'seg7')
('vid2', 'seg8')
('vid2', 'seg9')
('vid3', 'seg1')
('vid3', 'seg2')
('vid3', 'seg3')
('vid3', 'seg4')
('vid3', 'seg5')
('vid3', 'seg6')
('vid3', 'seg7')
('vid3', 'seg8')
('vid3', 'seg9')
('vid4', 'seg1')
('vid4', 'seg2')
('vid4', 'seg3')
('vid4', 'seg4')
('vid4', 'seg5')
('vid4', 'seg6')
('vid4', 'seg7')
('vid4', 'seg8')
('vid4', 'seg9')
('vid5', 'seg1')
('vid5', 'seg2')


In [6]:
import os
import pandas as pd

# ─── CONFIG ────────────────────────────────────────────────────────────────
# Ruta al CSV resultante de “majority” (sin duplicados de frame)
ann_path = "./annotations/per_modality/annotations_only_audio_majority.csv"
output_dir = "./annotations/cont/"
os.makedirs(output_dir, exist_ok=True)

# Frecuencia de muestreo de las anotaciones (60 Hz)
FPS = 60.0

# ─── 1. Cargar anotaciones “majority” ────────────────────────────────────────
df = pd.read_csv(ann_path)

# ─── 2. Generar segmentos contiguos para cada (video_id, segment, participant) ─
segments = []
for (vid, seg, pid), group in df.groupby(["video_id", "segment", "participant"]):
    # Ordenar por frame ascendente
    group = group.sort_values("frame").reset_index(drop=True)
    current_label = group.loc[0, "label"]
    start_frame   = group.loc[0, "frame"]

    # Recorrer cada fila para detectar cambios de etiqueta o saltos en el frame
    for i in range(1, len(group)):
        row        = group.loc[i]
        prev_frame = group.loc[i - 1, "frame"]

        # Si cambia el label o no_audio es frame consecutivo: cerramos segmento
        if (row["label"] != current_label) or (row["frame"] != prev_frame + 1):
            end_frame = prev_frame
            duration_s = (end_frame - start_frame + 1) / FPS

            segments.append({
                "video_id":    vid,
                "segment":     seg,
                "participant": pid,
                "start_frame": int(start_frame),
                "end_frame":   int(end_frame),
                "start_time":  start_frame / FPS,
                "end_time":    end_frame / FPS,
                "duration_s":  duration_s,
                "label":       int(current_label)
            })

            # Comenzar nuevo segmento
            start_frame   = row["frame"]
            current_label = row["label"]

    # Agregar el último segmento abierto
    end_frame = group.loc[len(group)-1, "frame"]
    duration_s = (end_frame - start_frame + 1) / FPS
    segments.append({
        "video_id":    vid,
        "segment":     seg,
        "participant": pid,
        "start_frame": int(start_frame),
        "end_frame":   int(end_frame),
        "start_time":  start_frame / FPS,
        "end_time":    end_frame / FPS,
        "duration_s":  duration_s,
        "label":       int(current_label)
    })

# ─── 3. Convertir a DataFrame y guardar ────────────────────────────────────────
df_segments = pd.DataFrame(segments)
out_path = os.path.join(output_dir, "annotations_only_audio_majority_cont.csv")
df_segments.to_csv(out_path, index=False)

print(f"✅ Guardado contiguous segments → {out_path}")


✅ Guardado contiguous segments → ./annotations/cont/annotations_only_audio_majority_cont.csv


In [7]:
import os
import pickle
import numpy as np
import pandas as pd
from glob import glob

# CONFIG
ANN_CSV = "./annotations/cont/annotations_with_audio_majority_cont.csv"
PKL_DIR = "./accelerometer/filtered"
OUT_CSV = "./features/fixed_really/with_audio/features_with_audio_2s.csv"
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)

FS = 56.0  # Sampling rate in Hz
WIN_SEC = 2.0
WIN_SAMPLES = int(WIN_SEC * FS)
FILTERED_COLS = ["accelX_filtered", "accelY_filtered", "accelZ_filtered"]

# Load annotations
df_ann = pd.read_csv(ANN_CSV)
df_ann = df_ann[df_ann["duration_s"] > 0.2]

# Load signals
signals = {}
for pkl in glob(os.path.join(PKL_DIR, "*.pkl")):
    pid = int(os.path.splitext(os.path.basename(pkl))[0])
    with open(pkl, "rb") as f:
        signals[pid] = pickle.load(f)

# Feature extractor for fixed 10s segments
def extract_fixed10s_features(df_sig, t0, t1):
    seg = df_sig[(df_sig["time"] >= t0) & (df_sig["time"] < t1)]
    X = seg[FILTERED_COLS].to_numpy()
    L, F = X.shape  # L = real samples, F = 3

    if L == 0:
        X2 = np.zeros((WIN_SAMPLES, F))
    elif L < WIN_SAMPLES:
        pad_rows = np.tile(X[-1], (WIN_SAMPLES - L, 1))
        X2 = np.vstack([X, pad_rows])
    else:
        X2 = X[:WIN_SAMPLES]

    feats = {}
    for i, col in enumerate(FILTERED_COLS):
        arr = X2[:, i]
        feats[f"{col}_mean"] = arr.mean()
        feats[f"{col}_var"] = arr.var()
        feats[f"{col}_energy"] = (arr ** 2).sum()
        d = np.diff(arr)
        feats[f"{col}_deriv_mean"] = d.mean() if len(d) > 0 else 0
        feats[f"{col}_deriv_std"] = d.std() if len(d) > 0 else 0

    feats["SMA"] = np.abs(X2).sum()
    if X2.shape[0] > 1:
        def safe_corr(x, y):
            if np.std(x) == 0 or np.std(y) == 0:
                return 0.0
            return np.corrcoef(x, y)[0, 1]

        feats["corr_xy"] = safe_corr(X2[:, 0], X2[:, 1])
        feats["corr_xz"] = safe_corr(X2[:, 0], X2[:, 2])
        feats["corr_yz"] = safe_corr(X2[:, 1], X2[:, 2])

    else:
        feats.update({k: 0 for k in ("corr_xy", "corr_xz", "corr_yz")})

    feats["padded_duration_s"] = WIN_SEC
    feats["real_sample_count"] = L
    feats["expected_sample_count"] = WIN_SAMPLES
    feats["padding_applied"] = int(L < WIN_SAMPLES)

    return feats

# Process segments into fixed-length chunks
records = []
for _, row in df_ann.iterrows():
    pid = row["participant"]
    if pid not in signals:
        continue

    df_sig = signals[pid]
    t0 = row["start_time"]
    t1 = row["end_time"]
    duration = t1 - t0
    num_chunks = int(np.ceil(duration / WIN_SEC))

    for i in range(num_chunks):
        start = t0 + i * WIN_SEC
        end = min(start + WIN_SEC, t1)
        feats = extract_fixed10s_features(df_sig, start, end)

        # Add metadata
        feats.update({
            "video_id": row["video_id"],
            "segment": f"{row['segment']}_chunk{i}",
            "participant": pid,
            "start_time": start,
            "end_time": end,
            "original_duration": duration,
            "label": row["label"]
        })
        records.append(feats)

# Save to CSV
df_out = pd.DataFrame.from_records(records)
df_out.to_csv(OUT_CSV, index=False)
print(f"✅ Saved {len(records)} → {OUT_CSV}")


✅ Saved 66789 → ./features/fixed/with_audio/features_with_audio_2s.csv


In [14]:
import pandas as pd

BASE_ANN_CONT = "./annotations/cont/annotations_with_audio_majority_cont.csv"

df = pd.read_csv(BASE_ANN_CONT)

# Sumar cuántos labels son 1
print((df["label"] == 1).sum())
print((df["label"] == 0).sum())


1270
2419
1008


In [16]:
import os
import numpy as np
import pandas as pd
import pickle
from glob import glob

# CONFIG
MODALITIES = ["with_audio", "only_audio", "no_audio"]
DURATIONS = [1, 2, 10]  # in seconds
BASE_ANN_CONT = "./annotations/cont/annotations_{modality}_majority_cont.csv"
PKL_DIR = "./accelerometer/filtered"
FILTERED_COLS = ["accelX_filtered", "accelY_filtered", "accelZ_filtered"]
FS = 56.0  # Hz

# Load signals
signals = {}
for pkl in glob(os.path.join(PKL_DIR, "*.pkl")):
    pid = int(os.path.splitext(os.path.basename(pkl))[0])
    with open(pkl, "rb") as f:
        signals[pid] = pickle.load(f)

def extract_centered_features_flexible(df_sig, center, duration):
    t0 = center - duration / 2
    t1 = center + duration / 2

    # Limitar a lo que realmente existe en la señal
    min_t, max_t = df_sig["time"].min(), df_sig["time"].max()
    t0 = max(t0, min_t)
    t1 = min(t1, max_t)

    seg = df_sig[(df_sig["time"] >= t0) & (df_sig["time"] < t1)]
    X = seg[FILTERED_COLS].to_numpy()
    L, F = X.shape
    feats = {}

    for i, col in enumerate(FILTERED_COLS):
        arr = X[:, i]
        feats[f"{col}_mean"] = arr.mean() if L > 0 else 0
        feats[f"{col}_var"] = arr.var() if L > 0 else 0
        feats[f"{col}_energy"] = (arr ** 2).sum() if L > 0 else 0
        d = np.diff(arr)
        feats[f"{col}_deriv_mean"] = d.mean() if len(d) > 0 else 0
        feats[f"{col}_deriv_std"] = d.std() if len(d) > 0 else 0

    feats["SMA"] = np.abs(X).sum() if L > 0 else 0

    if L > 1:
        def safe_corr(x, y):
            if np.std(x) == 0 or np.std(y) == 0:
                return 0.0
            return np.corrcoef(x, y)[0, 1]
        feats["corr_xy"] = safe_corr(X[:, 0], X[:, 1])
        feats["corr_xz"] = safe_corr(X[:, 0], X[:, 2])
        feats["corr_yz"] = safe_corr(X[:, 1], X[:, 2])
    else:
        feats.update({k: 0 for k in ("corr_xy", "corr_xz", "corr_yz")})

    feats["duration_s"] = t1 - t0
    feats["real_sample_count"] = L
    feats["start_time"] = t0
    feats["end_time"] = t1
    return feats


# Main loop over modality and duration
for modality in MODALITIES:
    ann_path = BASE_ANN_CONT.format(modality=modality)
    df_cont = pd.read_csv(ann_path)
    df_cont = df_cont[df_cont["duration_s"] >= 0.2]

    for dur in DURATIONS:
        df_dur = df_cont.copy()
        records = []

        for _, row in df_dur.iterrows():
            pid = row["participant"]
            if pid not in signals:
                continue

            df_sig = signals[pid]
            center = (row["start_time"] + row["end_time"]) / 2.0

            feats = extract_centered_features_flexible(df_sig, center, dur)

            feats.update({
                "video_id": row["video_id"],
                "segment": row["segment"],
                "participant": pid,
                "label": row["label"],
                "original_cont_duration": row["duration_s"]
            })
            records.append(feats)

        output_dir = f"./features/from_centered/{modality}/{dur}s"
        os.makedirs(output_dir, exist_ok=True)
        out_path = os.path.join(output_dir, f"annotations_{modality}_centered_{dur}s.csv")
        pd.DataFrame.from_records(records).to_csv(out_path, index=False)
        print(f"✅ Saved {len(records)} segments → {out_path}")


✅ Saved 3270 segments → ./features/from_centered/with_audio/1s/annotations_with_audio_centered_1s.csv
✅ Saved 3270 segments → ./features/from_centered/with_audio/2s/annotations_with_audio_centered_2s.csv
✅ Saved 3270 segments → ./features/from_centered/with_audio/10s/annotations_with_audio_centered_10s.csv
✅ Saved 3975 segments → ./features/from_centered/only_audio/1s/annotations_only_audio_centered_1s.csv
✅ Saved 3975 segments → ./features/from_centered/only_audio/2s/annotations_only_audio_centered_2s.csv
✅ Saved 3975 segments → ./features/from_centered/only_audio/10s/annotations_only_audio_centered_10s.csv
✅ Saved 3488 segments → ./features/from_centered/no_audio/1s/annotations_no_audio_centered_1s.csv
✅ Saved 3488 segments → ./features/from_centered/no_audio/2s/annotations_no_audio_centered_2s.csv
✅ Saved 3488 segments → ./features/from_centered/no_audio/10s/annotations_no_audio_centered_10s.csv


In [16]:
import os
import wandb
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble      import RandomForestClassifier
from sklearn.metrics       import classification_report, f1_score

def train():
    # 1) Start a W&B run
    run = wandb.init(project="rf-segment-classification", job_type="train")
    cfg = run.config

    # 2) Choose the right feature file
    base = "./features/fixed/no_audio"
    feat_file = os.path.join(base, f"features_no_audio_2s.csv")

    if not os.path.exists(feat_file):
        raise FileNotFoundError(f"Missing feature file: {feat_file}")

    # 3) Load data
    df = pd.read_csv(feat_file)
    drop_cols = [
    "video_id", "segment", "participant",
    "start_time", "end_time", "label",
    "padded_duration_s", "original_duration",
    "real_sample_count", "expected_sample_count",
    "padding_applied"
    ]
    feature_cols = [c for c in df.columns if c not in drop_cols]

    feature_cols = [c for c in df.columns if c not in drop_cols]

    X = df[feature_cols].fillna(0)
    y = df["label"]

    # 4) Stratified train/test split
    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=cfg.random_state
    )

    # 5) Train RandomForest
    clf = RandomForestClassifier(
        n_estimators   = cfg.n_estimators,
        max_depth      = cfg.max_depth,
        class_weight   = "balanced",
        random_state   = cfg.random_state,
        n_jobs         = -1
    )
    clf.fit(X_tr, y_tr)

    # 6) Evaluate
    y_pred = clf.predict(X_te)
    report = classification_report(y_te, y_pred, output_dict=True, zero_division=0)
    f1_macro = f1_score(y_te, y_pred, average="macro")

    # 7) Log all metrics
    metrics = {
        "test/accuracy":    report["accuracy"],
        "test/f1_macro":    f1_macro,
        "test/precision_0": report["0"]["precision"],
        "test/recall_0":    report["0"]["recall"],
        "test/f1_0":        report["0"]["f1-score"],
        "test/precision_1": report["1"]["precision"],
        "test/recall_1":    report["1"]["recall"],
        "test/f1_1":        report["1"]["f1-score"],
        "f1":               report["macro avg"]["f1-score"]
    }
    run.log(metrics)

    # 8) Log top‐10 feature importances
    importances = pd.Series(clf.feature_importances_, index=feature_cols)
    top10 = importances.nlargest(10)
    for feat, val in top10.items():
        run.log({f"feature_importance/{feat}": val})

    run.finish()


In [17]:
import wandb

sweep_config = {
    "method": "bayes",
    "metric": {"name": "test/f1_macro", "goal": "maximize"},
    "parameters": {
        "n_estimators":        {"values": [50, 100, 200]},
        "max_depth":           {"values": [None, 5, 10]},
        "random_state":        {"value": 42}
    }
}

sweep_id = wandb.sweep(sweep_config, project="rf-segment-classification")
print("Sweep ID:", sweep_id)


Create sweep with ID: qj7z584c
Sweep URL: https://wandb.ai/knezevicoluka-tu-delft/rf-segment-classification/sweeps/qj7z584c
Sweep ID: qj7z584c


In [18]:
# Run 12 trials (one for each window_size × hyperparam combination)
wandb.agent(sweep_id, function=train, count=12)


wandb: Agent Starting Run: n6xap2j1 with config:
wandb: 	max_depth: None
wandb: 	n_estimators: 200
wandb: 	random_state: 42


f1,▁
feature_importance/accelX_filtered_deriv_std,▁
feature_importance/accelX_filtered_mean,▁
feature_importance/accelX_filtered_var,▁
feature_importance/accelY_filtered_deriv_std,▁
feature_importance/accelY_filtered_mean,▁
feature_importance/accelY_filtered_var,▁
feature_importance/accelZ_filtered_deriv_std,▁
feature_importance/accelZ_filtered_mean,▁
feature_importance/accelZ_filtered_var,▁
feature_importance/corr_xy,▁


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: nr52jo5c with config:
wandb: 	max_depth: 10
wandb: 	n_estimators: 50
wandb: 	random_state: 42


f1,▁
feature_importance/accelX_filtered_deriv_std,▁
feature_importance/accelX_filtered_mean,▁
feature_importance/accelX_filtered_var,▁
feature_importance/accelY_filtered_deriv_std,▁
feature_importance/accelY_filtered_mean,▁
feature_importance/accelY_filtered_var,▁
feature_importance/accelZ_filtered_deriv_std,▁
feature_importance/accelZ_filtered_mean,▁
feature_importance/accelZ_filtered_var,▁
feature_importance/corr_xy,▁


wandb: Agent Starting Run: z4dncg49 with config:
wandb: 	max_depth: None
wandb: 	n_estimators: 100
wandb: 	random_state: 42


...

f1,▁
feature_importance/accelX_filtered_deriv_std,▁
feature_importance/accelX_filtered_mean,▁
feature_importance/accelX_filtered_var,▁
feature_importance/accelY_filtered_deriv_std,▁
feature_importance/accelY_filtered_mean,▁
feature_importance/accelY_filtered_var,▁
feature_importance/accelZ_filtered_deriv_std,▁
feature_importance/accelZ_filtered_mean,▁
feature_importance/accelZ_filtered_var,▁
feature_importance/corr_xy,▁


wandb: Agent Starting Run: kgl1xqic with config:
wandb: 	max_depth: 10
wandb: 	n_estimators: 50
wandb: 	random_state: 42


f1,▁
feature_importance/accelX_filtered_deriv_std,▁
feature_importance/accelX_filtered_mean,▁
feature_importance/accelX_filtered_var,▁
feature_importance/accelY_filtered_deriv_std,▁
feature_importance/accelY_filtered_mean,▁
feature_importance/accelY_filtered_var,▁
feature_importance/accelZ_filtered_deriv_std,▁
feature_importance/accelZ_filtered_mean,▁
feature_importance/accelZ_filtered_var,▁
feature_importance/corr_xy,▁


wandb: Agent Starting Run: uwk48t87 with config:
wandb: 	max_depth: 10
wandb: 	n_estimators: 50
wandb: 	random_state: 42


f1,▁
feature_importance/accelX_filtered_deriv_std,▁
feature_importance/accelX_filtered_mean,▁
feature_importance/accelX_filtered_var,▁
feature_importance/accelY_filtered_deriv_std,▁
feature_importance/accelY_filtered_mean,▁
feature_importance/accelY_filtered_var,▁
feature_importance/accelZ_filtered_deriv_std,▁
feature_importance/accelZ_filtered_mean,▁
feature_importance/accelZ_filtered_var,▁
feature_importance/corr_xy,▁
